In [1]:
#----------------------------------------------EDIT WITH CAUTION----------------------------------------------------

import numpy as np
import time

# Fill the matrix with random proportions
def rand_fill(TYPES):
    x = 0
    while x == 0:
        random = np.random.rand(TYPES)
        y = 0 - np.log(random)
        x = np.sum(y)
    pop = y / x
    return pop

# Fill the matrix with equal proportions
def consistent_fill(TYPES):
    return np.full(TYPES, 1.0 / TYPES)
    # Expect: 56.5% fairman, 43.5% easy rider
    #return np.array([.32, .02, .1, .02, .1, .02, .4, .02])


# Represents one generation in the population
def replicate(pop, oldpop, TYPES, matrix, mu, r):
    oldpop = np.copy(pop)
    fitness = np.zeros(TYPES)
    
    # For each strategy, calculate its payoff
    for i in range(TYPES):
        for j in range(TYPES):
            payoff_ij = matrix[i][j] * pop[j]
            fitness[i] += r * (matrix[i][i]) + (1 - r) * payoff_ij
    
    # Find the average fitness
    avefit = np.sum(fitness * pop)
    
    # For each strategy, change its proportion according to its payoff
    for i in range(TYPES):
        pop[i] = pop[i] * ((1 - mu) * fitness[i] / avefit) + mu / TYPES
    
    return pop, oldpop

# Detect if the population is stable
def detect_stable(oldpop, pop, threshold=1e-5):
    return np.all(np.abs(oldpop - pop) < threshold)

# Print an individual run
def print_run(pop, gen, totals, printgen):
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.001 else "_____" for prop in pop]
    if printgen:
        print(" ".join(formatted_proportions), "Generations:", gen)
    else:
            print(" ".join(formatted_proportions))
    totals += pop
    return totals

# Print the averaged results for each strategy
def print_results(sum_proportions, RUNS):
    mean_proportions = sum_proportions / RUNS
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.0001 else "0.00" for prop in mean_proportions]
    print("\033[1m\nMean Proportions Across Simulations: \033[0m")
    print(" ".join(formatted_proportions))

# Run the program
def run(matrix, mu=0.00, r=0.00, runs=10, maxgen=1000, printruns=True, printgen=False, randfill=True):
    TYPES=matrix.shape[0]
    np.random.seed(int(time.time()))
    
    totals = np.zeros(TYPES)
    sum_proportions = np.zeros(TYPES)
    
    # For each run, do this
    for i in range(runs):
        if (randfill):
            pop = rand_fill(TYPES)
        else:
            pop = consistent_fill(TYPES)
        oldpop = np.zeros(TYPES)
        gen = 0
        stable = False
        
        while not stable and gen < maxgen:
            pop, oldpop = replicate(pop, oldpop, TYPES, matrix, mu, r)
            stable = detect_stable(oldpop, pop)
            gen += 1
            
        if printruns:
            totals = print_run(pop, gen, totals, printgen)
        
        sum_proportions += pop
        
    print_results(sum_proportions, runs)

In [2]:

#----------------------------------------------RUNNING SIMULATIONS----------------------------------------------------
"""
Payoff Matrices
Symmetric games only - payoffs listed are only for player 1
"""
# Stag Hunt
staghunt = np.array([
    [4, 0],
    [3, 3]])

# Prisoner's Dilemma
prisonersdilemma = np.array([
    [2, 0],
    [3, 1]])

# Prisoners Dilemma with Punishment - Cnp (cooperate with no punish), Dnp (defect with no punish), E (cooperate then punish defect), A (coin flip then punish)
punishdilemma0 = np.array([
    [2, 0, 2, -1],
    [3, 1, 1, 0],
    [2, -1, 2, -1.5],
    [1.5, -0.5, 0.5, -1.5]])

punishdilemma = np.array([
    [12, 10, 12, 9],
    [13, 11, 11, 10],
    [12, 9, 12, 8.5],
    [11.5, 9.5, 10.5, 8.5]])

fullpunishdilemma = np.array([
    [2, 2, 0, 0, 0, 0, -2, -2], #Cnn (staunch utilitarian)
    [2, 2, 0, 0, -1, -1, -3, -3], #Cnp (enforcer)
    [1, 1, -1, -1, 0, 0, -2, -2], #Cpn (hates himself)
    [1, 1, -1, -1, -1, -1, -3, -3], #Cpp (not very intelliget)
    [3, 1, 3, 1, 1, -1, 1, -1], #Dnn (game theorist)
    [3, 1, 1, 1, 0, -2, 0, -2], #Dnp (hates himself)
    [2, 0, 2, 0, 1, -1, 1, -1], #Dpn (doesn't like people who don't know game theory)
    [2, 0, 2, 0, 0, -2, 0, -2]]) #Dpp (minimize opponent's payoff)

# Reduced Ultimatum Game
smallultimatum = np.array([
    [5.0, 2.5, 6.0, 5.0],
    [2.5, 5.0, 5.0, 0.0],
    [4.0, 5.0, 5.0, 1.5],
    [5.0, 0.0, 3.5, 5.0]])

# Mini Ultimatum Game
miniultimatum = np.array([
    [5.0, 0.5, 7.0, 2.5],
    [4.5, 0.0, 7.0, 2.5],
    [3.0, 3.0, 5.0, 5.0],
    [2.5, 2.5, 5.0, 5.0]])

# 7-3 Ultimatum Game
fullultimatum = np.array([
    [5.0, 5.0, 3.5, 1.5, 6.0, 6.0, 2.5, 2.5],
    [5.0, 5.0, 1.5, 0.0, 3.5, 3.5, 0.0, 0.0],
    [1.5, 3.5, 0.0, 0.0, 6.0, 6.0, 2.5, 2.5], 
    [3.5, 0.0, 0.0, 0.0, 3.5, 3.5, 0.0, 0.0],
    [4.0, 1.5, 4.0, 1.5, 5.0, 2.5, 5.0, 2.5], 
    [4.0, 1.5, 4.0, 1.5, 2.5, 0.0, 5.0, 0.0],
    [2.5, 0.0, 2.5, 0.0, 5.0, 5.0, 5.0, 2.5],
    [2.5, 0.0, 2.5, 0.0, 2.5, 0.0, 2.5, 0.0]])

# 9-1 Ultimatum Game
skyrmsfullultimatum = np.array([
    [10, 1, 1, 10, 14, 5, 5, 14],
    [9, 0, 0, 9, 9, 0, 0, 9],
    [9, 0, 0, 9, 14, 5, 5, 14],
    [10, 1, 1, 10, 9, 0, 0, 9],
    [6, 1, 6, 1, 10, 5, 10, 1],
    [5, 0, 5, 0, 5, 0, 5, 0],
    [5, 0, 5, 0, 10, 5, 10, 5],
    [6, 1, 6, 1, 5, 0, 5, 0]])


In [3]:
run(miniultimatum)

1.000 _____ _____ _____
1.000 _____ _____ _____
_____ _____ 0.203 0.797
1.000 _____ _____ _____
1.000 _____ _____ _____
1.000 _____ _____ _____
1.000 _____ _____ _____
1.000 _____ _____ _____
1.000 _____ _____ _____
1.000 _____ _____ _____

Mean Proportions Across Simulations: 
0.900 0.00 0.020 0.080
